In [1]:
import os

In [2]:
%pwd

'/mnt/d/Projects/Project Git repos/AutoVideoSectioning/Research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/mnt/d/Projects/Project Git repos/AutoVideoSectioning'

In [5]:
import logging
from pathlib import Path
logging.basicConfig(
    # filename='extract_data.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [6]:
# English Video

In [21]:
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, pipeline
from pydub import AudioSegment
import moviepy.editor as mp

# Function to extract audio from video
def extract_audio_from_video(video_path, audio_path):
    video = mp.VideoFileClip(str(video_path))  # Convert Path to string
    video.audio.write_audiofile(str(audio_path))  # Convert Path to string

# Function to transcribe audio using Hugging Face Transformers in 30-second chunks
def audio_to_text(audio_path, text_path):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
    model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english").to(device)

    audio = AudioSegment.from_file(audio_path)
    total_duration = len(audio)  # Total duration in milliseconds
    # total_duration = 120 * 1000  # 30 seconds in milliseconds
    chunk_duration = 30 * 1000  # 30 seconds in milliseconds
    text = ""

    for i in range(0, total_duration, chunk_duration):
        chunk = audio[i:i + chunk_duration]
        chunk.export("temp_chunk.wav", format="wav")
        
        # Load and preprocess the audio chunk
        speech_array, sampling_rate = librosa.load("temp_chunk.wav", sr=16_000)
        inputs = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)
        
        with torch.no_grad():
            logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits
        
        predicted_ids = torch.argmax(logits, dim=-1)
        chunk_text = processor.batch_decode(predicted_ids)[0]
        text += chunk_text + " "

    # Write the transcription to the text file
    with open(text_path, "w", encoding="utf-8") as file:
        file.write(text.strip())
    print("Text successfully written to", text_path)

# Paths
video_path = Path("Dataset/Videoinput/videoplayback.mp4")
audio_path = Path("Dataset/AudioExtracted/extracted_audio.wav")
text_path = Path("Dataset/Transcribe/english_translated_transcribed_text.txt")

# Extract audio from video
extract_audio_from_video(video_path, audio_path)

# Convert audio to text
audio_to_text(audio_path, text_path)

print("Text extracted and saved to", text_path)


MoviePy - Writing audio in Dataset/AudioExtracted/extracted_audio.wav


MoviePy - Done.


Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

Text successfully written to Dataset/Transcribe/english_translated_transcribed_text.txt
Text extracted and saved to Dataset/Transcribe/english_translated_transcribed_text.txt


# with time stamps

In [34]:
import torch
import pandas as pd
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from pydub import AudioSegment
import moviepy.editor as mp
from pathlib import Path

# Function to extract audio from video
def extract_audio_from_video(video_path, audio_path):
    video = mp.VideoFileClip(str(video_path))
    video.audio.write_audiofile(str(audio_path))

# Function to transcribe audio using Hugging Face Transformers in 30-second chunks
def audio_to_text(audio_path):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
    model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english").to(device)

    audio = AudioSegment.from_file(audio_path)
    total_duration = len(audio)
    chunk_duration = 30 * 1000  # 30 seconds in milliseconds
    chunks = []

    for i in range(0, total_duration, chunk_duration):
        chunk = audio[i:i + chunk_duration]
        chunk.export("temp_chunk.wav", format="wav")

        # Load and preprocess the audio chunk
        speech_array, sampling_rate = librosa.load("temp_chunk.wav", sr=16_000)
        inputs = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)

        with torch.no_grad():
            logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        chunk_text = processor.batch_decode(predicted_ids)[0]
        chunks.append(chunk_text)

    return chunks

# Paths
video_path = Path("Dataset/Videoinput/videoplayback.mp4")
audio_path = Path("Dataset/AudioExtracted/extracted_audio.wav")
output_excel_path = Path("Dataset/Timestamp/TimestampedText.xlsx")

# Extract audio from video
extract_audio_from_video(video_path, audio_path)

# Convert audio to text
transcriptions = audio_to_text(audio_path)

# Generate DataFrame
chunks_df = pd.DataFrame(transcriptions, columns=['Text'])

# Calculate time intervals
total_duration_secs = len(AudioSegment.from_file(audio_path)) / 1000  # Total duration in seconds
chunk_duration = 30  # Chunk duration in seconds
num_chunks = len(transcriptions)
time_from = [i * chunk_duration for i in range(num_chunks)]
time_to = [(i + 1) * chunk_duration if (i + 1) * chunk_duration < total_duration_secs else total_duration_secs for i in range(num_chunks)]

chunks_df['Time From (s)'] = time_from
chunks_df['Time To (s)'] = time_to

# Reorder columns
chunks_df = chunks_df[['Time From (s)', 'Time To (s)', 'Text']]

# Save DataFrame to Excel
chunks_df.to_excel(output_excel_path, index=False)

print("DataFrame saved to", output_excel_path)


MoviePy - Writing audio in Dataset/AudioExtracted/extracted_audio.wav


MoviePy - Done.


2024-06-10 16:59:34 - WARNING - /mnt/d/Projects/Project Git repos/AutoVideoSectioning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClass

DataFrame saved to Dataset/Timestamp/TimestampedText.xlsx


# rewrite by llama2

In [36]:
import pandas as pd
from llama2backend import generate_text
from pathlib import Path

# Define paths
input_excel_path = Path("Dataset/Timestamp/TimestampedText.xlsx")
output_excel_path = Path("Dataset/Timestamp/RewrittenTimestampedText.xlsx")

# Load the DataFrame
chunks_df = pd.read_excel(input_excel_path)

# Process each text cell using the LLM function
def rewrite_text(text):
    prompt = f"""rewrite that uses more proffesional English:
                {text}
             """
    rewritten_text = generate_text(prompt)
    return rewritten_text

# Apply the rewrite_text function to each cell in the 'Text' column
chunks_df['Rewritten_Text'] = chunks_df['Text'].apply(rewrite_text)

# Save the modified DataFrame to a new Excel file
chunks_df.to_excel(output_excel_path, index=False)

print("Rewritten DataFrame saved to", output_excel_path)


Llama.generate: prefix-match hit



Here's a more professional rewording of the text:
"Organizations are realizing that the trade is the future of digital businesses. By leveraging the power of AI, these companies can gain a competitive edge and stay ahead of the competition. The trade is not just a buzzword; it's a strategic imperative for any organization looking to thrive in today's fast-paced digital landscape. In five years, we will see a significant shift towards more sustainable and efficient digital businesses, with many companies adopting AI-driven trade as the cornerstone of their operations. The future of digital businesses is bright, and those that fail to adapt will be left behind."


llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =      36.74 ms /   154 runs   (    0.24 ms per token,  4192.07 tokens per second)
llama_print_timings: prompt eval time =    5443.75 ms /   101 tokens (   53.90 ms per token,    18.55 tokens per second)
llama_print_timings:        eval time =   11247.39 ms /   153 runs   (   73.51 ms per token,    13.60 tokens per second)
llama_print_timings:       total time =   15189.81 ms /   254 tokens
Llama.generate: prefix-match hit


 *  The agency model is when an influencer acts as an intermediary between brands and their audience. This means that the influencer will promote products or services on behalf of the brand, and earn money from the brand for doing so.
*  The affiliate model is when an influencer earns a commission by promoting products or services and including an affiliate link in their content. When someone clicks on this link and makes a purchase, the influencer earns a percentage of the sale as a commission.
*  The sponsored content model is when an influencer creates content that is paid for by a brand. This can take many forms, such as product reviews, tutorials, or even just a shoutout to the brand on social media. In return for creating this content, the influencer will often receive free products or money from the brand.
*  The business model of an influencer can also be a combination of these models. For example, an influencer may use affiliate links in their Instagram posts and also promote 


llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =      56.78 ms /   256 runs   (    0.22 ms per token,  4508.79 tokens per second)
llama_print_timings: prompt eval time =    3110.07 ms /    82 tokens (   37.93 ms per token,    26.37 tokens per second)
llama_print_timings:        eval time =   18928.34 ms /   255 runs   (   74.23 ms per token,    13.47 tokens per second)
llama_print_timings:       total time =   22490.67 ms /   337 tokens
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =       0.50 ms /     2 runs   (    0.25 ms per token,  4000.00 tokens per second)
llama_print_timings: prompt eval time =    4189.40 ms /   112 tokens (   37.41 ms per token,    26.73 tokens per second)
llama_print_timings:        eval time =      69.84 ms /     1 runs   (   69.84 ms per token,    14.32 tokens per second)
llama_print_timings:       total time =    4262.77 ms /   113 tokens
Llama.generate: prefix-match hit



llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =       0.73 ms /     3 runs   (    0.24 ms per token,  4092.77 tokens per second)
llama_print_timings: prompt eval time =    4982.49 ms /   131 tokens (   38.03 ms per token,    26.29 tokens per second)
llama_print_timings:        eval time =     143.48 ms /     2 runs   (   71.74 ms per token,    13.94 tokens per second)
llama_print_timings:       total time =    5130.91 ms /   133 tokens
Llama.generate: prefix-match hit



See changes made:
1. Changed "your" to "your" (lowercase) to match the original text.
2. Changed "demandl" to "demand" (no change needed).
3. Changed "maverage" to "average" (correct spelling).
4. Changed "goprintall-yu" to "go online" (better phrasing).
5. Changed "kit" to "it" (better pronoun usage).
6. Changed "fuckingal" to "fucking all" (more professional language).
7. Changed "qschool" to "quitting school" (better phrasing).
8. Changed "gostarto" to "start an online business" (better phrasing).
9. Changed "orecomon" to "common" (correct spelling).
10. Changed "hell" to "a common" (better phrasing).


llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =      44.67 ms /   218 runs   (    0.20 ms per token,  4880.12 tokens per second)
llama_print_timings: prompt eval time =    1761.21 ms /    48 tokens (   36.69 ms per token,    27.25 tokens per second)
llama_print_timings:        eval time =   16517.72 ms /   217 runs   (   76.12 ms per token,    13.14 tokens per second)
llama_print_timings:       total time =   18654.32 ms /   265 tokens


Rewritten DataFrame saved to Dataset/Timestamp/RewrittenTimestampedText.xlsx


In [43]:
import pandas as pd
from llama2backend import generate_text
from pathlib import Path

# Define paths
input_excel_path = Path("Dataset/Timestamp/RewrittenTimestampedText.xlsx")
output_excel_path = Path("Dataset/Timestamp/TopicsCoveredRewrittenTimestampedText.xlsx")

# Load the DataFrame
chunks_df = pd.read_excel(input_excel_path)

# # Process each text cell using the LLM function for extracting topics
# def extract_topics(text):
#     prompt = f"""Just give the main  Topics and subtopics names headings covered in the following text in bullet points:
#                 {text}
#              """
#     topics = generate_text(prompt)
#     return topics

# Process each text cell using the LLM function for extracting topics
def extract_topics(text):
    prompt = f"""Just summarize in bulletpoint headings as Topics and subtopics:
                {text}
             """
    topics = generate_text(prompt)
    return topics

# Concatenate 'Rewritten_Text' and 'Topics_Covered' into a new column 'Concatenated_Text'
chunks_df['Concatenated_Text'] = chunks_df.apply(lambda row: f"{row['Rewritten_Text']} Topics Covered: {row['Text']}", axis=1)

# Apply the extract_topics function to each cell in the 'Rewritten_Text' column
chunks_df['Topics_Covered'] = chunks_df['Concatenated_Text'].apply(extract_topics)

# Save the modified DataFrame to a new Excel file
chunks_df.to_excel(output_excel_path, index=False)

print("Rewritten DataFrame with topics covered and concatenated text saved to", output_excel_path)


Llama.generate: prefix-match hit


 + Why the trade is the future of digital businesses 
                   - Leveraging AI for a competitive edge 
                    + The trade is not just a buzzword 
                      - It's a strategic imperative for any organization looking to thrive in today's fast-paced digital landscape.
                        + How the trade will shape digital businesses in five years 
                           - A significant shift towards more sustainable and efficient digital businesses 
                             + Many companies adopting AI-driven trade as the cornerstone of their operations.
                                   + The future of digital businesses is bright, and those that fail to adapt will be left behind.


llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =      34.70 ms /   154 runs   (    0.23 ms per token,  4437.40 tokens per second)
llama_print_timings: prompt eval time =   12754.11 ms /   268 tokens (   47.59 ms per token,    21.01 tokens per second)
llama_print_timings:        eval time =   11687.94 ms /   153 runs   (   76.39 ms per token,    13.09 tokens per second)
llama_print_timings:       total time =   22094.02 ms /   421 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =       0.28 ms /     1 runs   (    0.28 ms per token,  3571.43 tokens per second)
llama_print_timings: prompt eval time =   12989.93 ms /   343 tokens (   37.87 ms per token,    26.41 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   12999.23 ms /   344 

 * Grandparents for starting income sports
                     + Little trying to sell every video at this point (years away)
                         - Across the world, there's gonna blow off doesn't matter anymore (consumers get on overlooking of people trying to solve in just lose interest in people jut gonna lose their nets)



llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =      17.77 ms /    76 runs   (    0.23 ms per token,  4277.35 tokens per second)
llama_print_timings: prompt eval time =    3603.39 ms /    96 tokens (   37.54 ms per token,    26.64 tokens per second)
llama_print_timings:        eval time =    5678.89 ms /    76 runs   (   74.72 ms per token,    13.38 tokens per second)
llama_print_timings:       total time =    9407.12 ms /   172 tokens
Llama.generate: prefix-match hit


...

 * Online Businesses
                  + Successful online businesses within the past three years
                  + Making six figures within fourty-five years
                  + People who've been in the industry over ten years and making a hundred and fifty thousand clots
                  + Those who've ben there over fifty years
                  + Quatmill anapomberbs like thirty prints threehundred thousand dollars a year
                  + He's been in the industry for nine to ten years
                  + Doesn't matter how long you've been in the industry as long as the environment and city are good





llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =      29.04 ms /   138 runs   (    0.21 ms per token,  4752.88 tokens per second)
llama_print_timings: prompt eval time =    4962.07 ms /   136 tokens (   36.49 ms per token,    27.41 tokens per second)
llama_print_timings:        eval time =   10173.71 ms /   138 runs   (   73.72 ms per token,    13.56 tokens per second)
llama_print_timings:       total time =   15359.58 ms /   274 tokens
Llama.generate: prefix-match hit


 * Demand and average national demand
                   + Your demand (lowercase)
                   + Maverage (correct spelling)
                   + National demand





llama_print_timings:        load time =     318.32 ms
llama_print_timings:      sample time =       8.41 ms /    38 runs   (    0.22 ms per token,  4517.36 tokens per second)
llama_print_timings: prompt eval time =    9981.32 ms /   269 tokens (   37.11 ms per token,    26.95 tokens per second)
llama_print_timings:        eval time =    2747.37 ms /    37 runs   (   74.25 ms per token,    13.47 tokens per second)
llama_print_timings:       total time =   12790.62 ms /   306 tokens


Rewritten DataFrame with topics covered and concatenated text saved to Dataset/Timestamp/TopicsCoveredRewrittenTimestampedText.xlsx


# Hindi Video

In [18]:
# %%time
# import torch
# import librosa
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, pipeline
# from pydub import AudioSegment
# import moviepy.editor as mp

# # Function to extract audio from video
# def extract_audio_from_video(video_path, audio_path):
#     video = mp.VideoFileClip(str(video_path))  # Convert Path to string
#     video.audio.write_audiofile(str(audio_path))  # Convert Path to string

# # Function to transcribe audio using Hugging Face Transformers in 30-second chunks
# def audio_to_text(audio_path, text_path):
#     device = "cuda:0" if torch.cuda.is_available() else "cpu"
#     processor = Wav2Vec2Processor.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi")
#     model = Wav2Vec2ForCTC.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi").to(device)

#     audio = AudioSegment.from_file(audio_path)
#     # total_duration = len(audio)  # Total duration in milliseconds
#     total_duration = 10 * 30  * 1000  # 30 seconds in milliseconds
#     chunk_duration = 30 * 1000  # 30 seconds in milliseconds
#     text = ""

#     for i in range(0, total_duration, chunk_duration):
#         chunk = audio[i:i + chunk_duration]
#         chunk.export("temp_chunk.wav", format="wav")
        
#         # Load and preprocess the audio chunk
#         speech_array, sampling_rate = librosa.load("temp_chunk.wav", sr=16_000)
#         inputs = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)
        
#         with torch.no_grad():
#             logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits
        
#         predicted_ids = torch.argmax(logits, dim=-1)
#         chunk_text = processor.batch_decode(predicted_ids)[0]
#         text += chunk_text + " "

#     # Write the transcription to the text file
#     with open(text_path, "w", encoding="utf-8") as file:
#         file.write(text.strip())
#     print("Text successfully written to", text_path)

# # Paths
# video_path = Path("Dataset/Videoinput/MERN JUNE 09-06-2024.mp4")
# audio_path = Path("Dataset/AudioExtracted/extracted_audio.wav")
# text_path = Path("Dataset/Transcribe/transcribed_text.txt")

# # Extract audio from video
# # extract_audio_from_video(video_path, audio_path)

# # Convert audio to text
# audio_to_text(audio_path, text_path)

# print("Text extracted and saved to", text_path)


Some weights of the model checkpoint at theainerd/Wav2Vec2-large-xlsr-hindi were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at theainerd/Wav2Vec2-large-xlsr-hindi and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably T

Text successfully written to Dataset/Transcribe/transcribed_text.txt
Text extracted and saved to Dataset/Transcribe/transcribed_text.txt
CPU times: user 6.41 s, sys: 993 ms, total: 7.4 s
Wall time: 8.27 s


# Hindi to english

In [19]:
# %%time
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # Paths
# hindi_text_path = Path("Dataset/Transcribe/transcribed_text.txt")
# english_text_path = Path("Dataset/Transcribe/english_translated_transcribed_text.txt")

# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("snehalyelmati/mt5-hindi-to-english")
# model = AutoModelForSeq2SeqLM.from_pretrained("snehalyelmati/mt5-hindi-to-english")

# # Function to translate Hindi text to English in chunks
# def translate_text_in_chunks(hindi_text_path, english_text_path, chunk_size=500):
#     with open(hindi_text_path, "r", encoding="utf-8") as file:
#         hindi_text = file.read()
    
#     # Split text into chunks
#     chunks = [hindi_text[i:i + chunk_size] for i in range(0, len(hindi_text), chunk_size)]
    
#     translated_text = ""
    
#     for chunk in chunks:
#         inputs = tokenizer.encode(chunk, return_tensors="pt", max_length=512, truncation=True)
#         outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
#         translated_chunk = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         translated_text += translated_chunk + " "
    
#     # Write the translated text to the text file
#     with open(english_text_path, "w", encoding="utf-8") as file:
#         file.write(translated_text.strip())
    
#     print("Translated text successfully written to", english_text_path)

# # Translate the text
# translate_text_in_chunks(hindi_text_path, english_text_path)


Translated text successfully written to Dataset/Transcribe/english_translated_transcribed_text.txt
CPU times: user 1min, sys: 236 ms, total: 1min
Wall time: 8.66 s
